# 멀티 호라이즌 섹터 예측 + 호라이즌별 인더스트리 클러스터링

## 워크플로우
1. 멀티 호라이즌 예측 (1d, 3d, 1w, 1m, 1q, 6m, 1y)
2. **각 호라이즌별로 독립적으로:**
   - Top-3 섹터 선정
   - 해당 섹터 내 인더스트리 클러스터링 (KMeans, 4개 클러스터)
   - 결과를 `Data_set/Cluster_Results/`에 CSV로 저장
3. 전체 결과 JSON 저장

## 1. 설정 및 데이터 로딩

In [1]:
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from src import MultiHorizonPredictor, IndustryClusterer

print("모듈 로딩 완료")

모듈 로딩 완료


In [2]:
# 전체 데이터셋 로드 (Industry 컬럼이 포함된 주식 단위 데이터)
df_raw = pd.read_csv('Data_set/stock_features_clean.csv', parse_dates=['Date'])

print(f"데이터셋 로드: {len(df_raw):,} 행")
print(f"날짜 범위: {df_raw['Date'].min().date()} ~ {df_raw['Date'].max().date()}")
print(f"컬럼: {df_raw.columns.tolist()}")
print(f"\n섹터: {sorted(df_raw['Sector'].unique())}")
print(f"산업 수: {df_raw['Industry'].nunique()}")
print(f"기업 수: {df_raw['Company'].nunique()}")

데이터셋 로드: 603,359 행
날짜 범위: 2020-11-27 ~ 2026-01-09
컬럼: ['Date', 'Company', 'Sector', 'Industry', 'Open', 'High', 'Low', 'Close', 'Volume', 'Daily_Return_raw', 'Daily_Return_calc', 'Cum_Return', 'Return_1M', 'Return_3M', 'Return_6M', 'MA_5', 'MA_20', 'MA_60', 'Volatility_20d', 'Drawdown', 'MDD', 'DD_Short', 'Vol_MA_20', 'Vol_Ratio', 'Vol_Std_20', 'Vol_Z_Score', 'Log_Volume', 'Log_Volume_W', 'RSI_14', 'BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width', 'Dividends', 'Stock Splits', 'Daily_Return', 'Cum_Max', 'Prev_Close', 'Gap', 'Gap_Pct', 'Is_Extreme_Change']

섹터: ['Basic Materials', 'Communication Services', 'Consumer Cyclical', 'Consumer Defensive', 'Energy', 'Financial Services', 'Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']
산업 수: 100
기업 수: 481


## 2. 파라미터 설정

In [3]:
# 예측 파라미터
PREDICTION_DATE = pd.Timestamp('2026-01-16')  # 원하는 예측 기준일로 변경
TRAIN_YEARS = 4  # 학습 데이터 연도 수

# 모델 파라미터 (백테스트 최적화 결과)
ALPHA = 0.6  # 하이브리드 모델 가중치
GAMMA = 0.5  # 랭킹에서 신뢰도 가중치
TOP_K = 3    # 호라이즌별 상위 섹터 수

# 인더스트리 클러스터링 파라미터
N_CLUSTERS = 5  # 클러스터 개수 (5개로 증가)

# 호라이즌별 lookback 기간 (일)
# 최소 60일 이상 필요 (통계적 신뢰성을 위한 충분한 거래일 확보)
HORIZON_LOOKBACK_MAP = {
    '1d': 60,     # 60일 (단기 트렌드)
    '3d': 75,     # 75일 (단기 트렌드)
    '1w': 90,     # 90일 (중단기 트렌드)
    '1m': 105,    # 105일 (중기 트렌드)
    '1q': 120,    # 120일 (중장기 트렌드)
    '6m': 150,    # 150일 (장기 트렌드)
    '1y': 180     # 180일 (장기 트렌드)
}

# 출력 디렉토리
OUTPUT_DIR = 'Data_set/Cluster_Results'
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("파라미터:")
print(f"  예측 기준일: {PREDICTION_DATE.date()}")
print(f"  학습 연도: {TRAIN_YEARS}")
print(f"  Alpha: {ALPHA}")
print(f"  Gamma: {GAMMA}")
print(f"  Top-K: {TOP_K}")
print(f"  클러스터 수: {N_CLUSTERS} (5개 - 세밀한 분류)")
print(f"\n호라이즌별 특성 계산 기간 (차등화):")
for horizon, days in HORIZON_LOOKBACK_MAP.items():
    print(f"  {horizon}: {days}일 (~{int(days*0.7)} 거래일)")
print(f"\n출력 디렉토리: {OUTPUT_DIR}")

파라미터:
  예측 기준일: 2026-01-16
  학습 연도: 4
  Alpha: 0.6
  Gamma: 0.5
  Top-K: 3
  클러스터 수: 5 (5개 - 세밀한 분류)

호라이즌별 특성 계산 기간 (차등화):
  1d: 60일 (~42 거래일)
  3d: 75일 (~52 거래일)
  1w: 90일 (~62 거래일)
  1m: 105일 (~73 거래일)
  1q: 120일 (~84 거래일)
  6m: 150일 (~105 거래일)
  1y: 180일 (~125 거래일)

출력 디렉토리: Data_set/Cluster_Results


## 3. 멀티 호라이즌 예측을 위한 데이터 준비

In [4]:
# 섹터 단위로 집계 (일별 평균)
sector_df = df_raw.groupby(['Date', 'Sector'], as_index=False).agg({
    'Close': 'mean'
}).sort_values(['Sector', 'Date'])

print(f"섹터 집계: {len(sector_df):,} 행")

# 섹터 목록 추출
sectors = sorted(sector_df['Sector'].unique())
print(f"섹터: {sectors}")

섹터 집계: 14,135 행
섹터: ['Basic Materials', 'Communication Services', 'Consumer Cyclical', 'Consumer Defensive', 'Energy', 'Financial Services', 'Healthcare', 'Industrials', 'Real Estate', 'Technology', 'Utilities']


## 4. 멀티 호라이즌 예측

In [5]:
# 예측기 초기화
predictor = MultiHorizonPredictor(
    alpha=ALPHA,
    gamma=GAMMA,
    top_k=TOP_K
)

# 멀티 호라이즌 예측 실행
multi_horizon_results = predictor.predict_all_horizons(
    df=sector_df,
    prediction_date=PREDICTION_DATE,
    sectors=sectors,
    train_years=TRAIN_YEARS
)

22:00:12 - cmdstanpy - INFO - Chain [1] start processing



2026-01-16 기준 멀티 호라이즌 예측
학습 데이터: 2022-01-18 ~ 2026-01-09
호라이즌: ['1d', '3d', '1w', '1m', '1q', '6m', '1y']

[1d] 1일 호라이즌 예측 중...


22:00:12 - cmdstanpy - INFO - Chain [1] done processing


22:00:12 - cmdstanpy - INFO - Chain [1] start processing


22:00:12 - cmdstanpy - INFO - Chain [1] done processing


...

22:00:45 - cmdstanpy - INFO - Chain [1] start processing


22:00:45 - cmdstanpy - INFO - Chain [1] done processing


22:00:45 - cmdstanpy - INFO - Chain [1] start processing


22:00:46 - cmdstanpy - INFO - Chain [1] done processing


  상위 3개 섹터: ['Industrials', 'Technology', 'Financial Services']

멀티 호라이즌 예측 완료


In [6]:
# 호라이즌별 상위 섹터 출력
print("\n" + "="*80)
print("호라이즌별 상위 섹터")
print("="*80)
for horizon, result in multi_horizon_results.items():
    print(f"\n{horizon} ({result['horizon_days']}일):")
    for i, sector in enumerate(result['top_sectors'], 1):
        pred = result['predictions'][sector]
        conf = result['confidences'][sector]
        score = result['ranking_scores'][sector]
        print(f"  {i}. {sector:25s} | 예측: {pred:7.2%} | 신뢰도: {conf:.3f} | 점수: {score:6.2f}")
print("="*80)


호라이즌별 상위 섹터

1d (1일):
  1. Healthcare                | 예측:   4.41% | 신뢰도: 0.520 | 점수:   2.28
  2. Energy                    | 예측:   2.65% | 신뢰도: 0.487 | 점수:   1.12
  3. Technology                | 예측:   0.74% | 신뢰도: 0.568 | 점수:   0.78

3d (3일):
  1. Industrials               | 예측:   1.43% | 신뢰도: 0.581 | 점수:   2.81
  2. Consumer Cyclical         | 예측:   0.73% | 신뢰도: 0.480 | 점수:   1.05
  3. Energy                    | 예측:   0.77% | 신뢰도: 0.396 | 점수:   0.55

1w (7일):
  1. Industrials               | 예측:   2.10% | 신뢰도: 0.631 | 점수:   2.44
  2. Consumer Cyclical         | 예측:   1.30% | 신뢰도: 0.517 | 점수:   0.94
  3. Communication Services    | 예측:   1.95% | 신뢰도: 0.387 | 점수:   0.87

1m (30일):
  1. Technology                | 예측:   6.32% | 신뢰도: 0.553 | 점수:   1.92
  2. Industrials               | 예측:   4.50% | 신뢰도: 0.543 | 점수:   1.20
  3. Communication Services    | 예측:   5.27% | 신뢰도: 0.407 | 점수:   0.72

1q (90일):
  1. Communication Services    | 예측:   6.71% | 신뢰도: 0.386 | 점수:   1.32
  2. Financi

## 5. 각 호라이즌별 인더스트리 클러스터링

In [7]:
# 클러스터러 초기화
clusterer = IndustryClusterer(
    n_clusters=N_CLUSTERS,
    random_state=42
)

# 각 호라이즌별로 클러스터링 수행
all_clustering_results = {}

date_str = PREDICTION_DATE.strftime('%Y%m%d')

for horizon_name, horizon_result in multi_horizon_results.items():
    print("\n" + "="*80)
    print(f"[{horizon_name}] 호라이즌 클러스터링 시작")
    print("="*80)
    
    # 해당 호라이즌의 Top-3 섹터
    top_sectors = horizon_result['top_sectors']
    print(f"선정된 섹터: {top_sectors}")
    
    # 호라이즌별 lookback 기간 가져오기
    lookback_days = HORIZON_LOOKBACK_MAP[horizon_name]
    print(f"특성 계산 기간: {lookback_days}일")
    
    # 클러스터링 수행
    clustering_result = clusterer.run_full_pipeline(
        df=df_raw,
        selected_sectors=top_sectors,
        lookback_days=lookback_days,  # 호라이즌별 lookback 사용
        end_date=PREDICTION_DATE
    )
    
    # 결과 저장
    all_clustering_results[horizon_name] = clustering_result
    
    # CSV 파일로 저장
    # 1. 전체 산업 특성
    df_industry = clustering_result['industry_features'].copy()
    df_industry.insert(0, 'Horizon', horizon_name)  # Horizon 컬럼 추가
    industry_filename = f"{OUTPUT_DIR}/{date_str}_{horizon_name}_industry_features.csv"
    df_industry.to_csv(industry_filename, index=False, encoding='utf-8-sig')
    print(f"\n저장: {industry_filename} ({len(df_industry)}개 산업)")
    
    # 2. 클러스터별 파일
    for cluster_name, df_cluster in clustering_result['by_cluster'].items():
        df_cluster_copy = df_cluster.copy()
        df_cluster_copy.insert(0, 'Horizon', horizon_name)  # Horizon 컬럼 추가
        cluster_filename = f"{OUTPUT_DIR}/{date_str}_{horizon_name}_{cluster_name}.csv"
        df_cluster_copy.to_csv(cluster_filename, index=False, encoding='utf-8-sig')
        print(f"저장: {cluster_filename} ({len(df_cluster_copy)}개 산업)")
    
    print(f"\n[{horizon_name}] 완료")

print("\n" + "="*80)
print("모든 호라이즌 클러스터링 완료")
print("="*80)


[1d] 호라이즌 클러스터링 시작
선정된 섹터: ['Healthcare', 'Energy', 'Technology']
특성 계산 기간: 60일

산업 특성 추출 중...
기간: 2025-11-17 ~ 2026-01-16
섹터: ['Healthcare', 'Energy', 'Technology']
산업 수: 26
26개 산업의 특성 추출 완료



KMeans 클러스터링 완료
클러스터 수: 5
클러스터 분포: {0: 9, 1: 2, 2: 6, 3: 1, 4: 8}

클러스터 프로파일
         Return_Period  Volatility_20d     MDD  Sharpe_Ratio
cluster                                                     
0              -0.0147          0.1872 -0.0746       -0.8455
1               0.2803          0.4616 -0.0955        3.9583
2               0.0462          0.3465 -0.0905        0.9662
3              -0.1332          0.3219 -0.2222       -2.9429
4               0.0761          0.1629 -0.0444        2.9394

클러스터 재매핑 (샤프 비율 구간 기반):
  Cluster 0: Sharpe=  0.61 | Return= 28.03% | Vol=46.16% | MDD= -9.55% (고수익·고위험 (Sharpe >= 1.0 || 고변동성))
  Cluster 1: Sharpe=  0.47 | Return= 10.08% | Vol=21.61% | MDD= -5.39% (Sharpe >= 1.5 && Vol < 40%)
  Cluster 2: Sharpe=  0.19 | Return=  3.10% | Vol=16.65% | MDD= -6.07% (0.5 <= Sharpe < 1.5 && Vol < 25%)
  Cluster 3: Sharpe=  0.07 | Return=  2.02% | Vol=29.70% | MDD= -7.54% (Sharpe >= 0 (저수익))
  Cluster 4: Sharpe= -0.15 | Return= -3.29% | Vol=21.39% | MDD= -9.3


KMeans 클러스터링 완료
클러스터 수: 5
클러스터 분포: {0: 14, 1: 5, 2: 9, 3: 2, 4: 1}

클러스터 프로파일
         Return_Period  Volatility_20d     MDD  Sharpe_Ratio
cluster                                                     
0               0.0665          0.1672 -0.0720        1.2215
1               0.2257          0.2181 -0.0805        3.2863
2              -0.0794          0.1601 -0.1477       -1.9743
3              -0.0767          0.3547 -0.2875       -0.6089
4               0.0823          0.4159 -0.1149        0.7206

클러스터 재매핑 (샤프 비율 구간 기반):
  Cluster 1: Sharpe=  0.88 | Return= 16.65% | Vol=18.85% | MDD= -7.55% (Sharpe >= 1.5 && Vol < 40%)
  Cluster 2: Sharpe=  0.26 | Return=  4.43% | Vol=16.81% | MDD= -7.09% (0.5 <= Sharpe < 1.5 && Vol < 25%)
  Cluster 3: Sharpe=  0.10 | Return=  2.84% | Vol=28.43% | MDD=-12.29% (Sharpe >= 0 (저수익))
  Cluster 4: Sharpe= -0.46 | Return= -8.11% | Vol=17.70% | MDD=-15.94% (Sharpe < 0)

클러스터 해석

CLUSTER_1: 강력 매수
  샤프: 0.88 | 수익률: 16.65% | 변동성: 18.85% | MDD: -7.55%
  산업 (11

## 6. 호라이즌별 클러스터 프로파일 요약

In [8]:
# 각 호라이즌별 클러스터 프로파일 출력
print("\n" + "="*80)
print("호라이즌별 클러스터 프로파일 요약")
print("="*80)

for horizon_name in multi_horizon_results.keys():
    clustering_result = all_clustering_results[horizon_name]
    
    print(f"\n{'='*80}")
    print(f"[{horizon_name}] - Top 섹터: {', '.join(multi_horizon_results[horizon_name]['top_sectors'])}")
    print(f"{'='*80}")
    
    cluster_profile = clustering_result['cluster_profile']
    print(cluster_profile)
    
    print(f"\n클러스터별 산업 수:")
    for cluster_name, df_cluster in clustering_result['by_cluster'].items():
        interp = clustering_result['interpretations'][cluster_name]
        print(f"  {cluster_name}: {len(df_cluster):2d}개 산업 - {interp['label']}")

print("\n" + "="*80)


호라이즌별 클러스터 프로파일 요약

[1d] - Top 섹터: Healthcare, Energy, Technology
         Return_Period  Volatility_20d       MDD  Sharpe_Ratio
cluster                                                       
0             0.280294        0.461644 -0.095540      0.607165
1             0.100808        0.216099 -0.053889      0.466489
2             0.030975        0.166539 -0.060669      0.185991
3             0.020225        0.297031 -0.075410      0.068092
4            -0.032934        0.213900 -0.093801     -0.153967

클러스터별 산업 수:
  cluster_0:  2개 산업 - 고수익·고위험
  cluster_1:  8개 산업 - 강력 매수
  cluster_2:  2개 산업 - 안정형/중립
  cluster_3:  4개 산업 - 저수익·고위험
  cluster_4: 10개 산업 - 관망/주의

[3d] - Top 섹터: Industrials, Consumer Cyclical, Energy
         Return_Period  Volatility_20d       MDD  Sharpe_Ratio
cluster                                                       
1             0.126979        0.205554 -0.060079      0.617738
2             0.043711        0.194232 -0.059413      0.225045
3             0.059580     

## 7. 호라이즌별 상위 산업 출력

In [9]:
# 각 호라이즌별로 강력 매수 클러스터(cluster_1)의 상위 산업 출력
print("\n" + "="*80)
print("호라이즌별 추천 산업 (강력 매수 - cluster_1)")
print("="*80)

for horizon_name in multi_horizon_results.keys():
    clustering_result = all_clustering_results[horizon_name]
    
    print(f"\n[{horizon_name}]")
    print("-" * 80)
    
    # cluster_1 (강력 매수) 찾기
    if 'cluster_1' in clustering_result['by_cluster']:
        df_best = clustering_result['by_cluster']['cluster_1']
        interp = clustering_result['interpretations']['cluster_1']
        
        print(f"클러스터: cluster_1 ({interp['label']})")
        print(f"샤프 비율: {interp['sharpe_ratio']:.2f}")
        print(f"\n상위 5개 산업:")
        
        for i, (idx, row) in enumerate(df_best.head(5).iterrows(), 1):
            print(f"  {i}. {row['Industry']:40s} | 섹터: {row['Sector']:20s} | 수익률: {row['Return_Period']:7.2%} | 샤프: {row['Sharpe_Ratio']:5.2f}")
    else:
        print("강력 매수 클러스터가 없습니다.")

print("\n" + "="*80)


호라이즌별 추천 산업 (강력 매수 - cluster_1)

[1d]
--------------------------------------------------------------------------------
클러스터: cluster_1 (강력 매수)
샤프 비율: 0.47

상위 5개 산업:
  1. Scientific & Technical Instruments       | 섹터: Technology           | 수익률:  14.36% | 샤프:  5.63
  2. Healthcare Plans                         | 섹터: Healthcare           | 수익률:  12.69% | 샤프:  3.54
  3. Oil & Gas Equipment & Services           | 섹터: Energy               | 수익률:  11.79% | 샤프:  2.29
  4. Semiconductors                           | 섹터: Technology           | 수익률:  11.37% | 샤프:  2.41
  5. Drug Manufacturers - Specialty & Generic | 섹터: Healthcare           | 수익률:  10.08% | 샤프:  4.50

[3d]
--------------------------------------------------------------------------------
클러스터: cluster_1 (강력 매수)
샤프 비율: 0.62

상위 5개 산업:
  1. Trucking                                 | 섹터: Industrials          | 수익률:  26.15% | 샤프:  5.54
  2. Specialty Retail                         | 섹터: Consumer Cyclical    | 수익률:  25.98% | 샤프:  6.58

## 9. 최종 요약

In [10]:
print("\n" + "="*80)
print("파이프라인 최종 요약")
print("="*80)
print(f"예측 기준일: {PREDICTION_DATE.date()}")
print(f"\n멀티 호라이즌 예측:")
print(f"  분석된 호라이즌: {len(multi_horizon_results)}개")
print(f"  호라이즌별 Top 섹터 수: {TOP_K}개")

print(f"\n호라이즌별 클러스터링 결과:")
for horizon_name in multi_horizon_results.keys():
    clustering_result = all_clustering_results[horizon_name]
    top_sectors = multi_horizon_results[horizon_name]['top_sectors']
    n_industries = len(clustering_result['industry_features'])
    
    print(f"\n  [{horizon_name}]:")
    print(f"    선정 섹터: {', '.join(top_sectors)}")
    print(f"    분석된 산업: {n_industries}개")
    print(f"    클러스터: {N_CLUSTERS}개")
    
    # 클러스터별 산업 수
    cluster_dist = {}
    for cluster_name, df_cluster in clustering_result['by_cluster'].items():
        cluster_dist[cluster_name] = len(df_cluster)
    print(f"    클러스터 분포: {cluster_dist}")

print(f"\n저장된 파일:")
print(f"  출력 디렉토리: {OUTPUT_DIR}")
print(f"  CSV 파일: {len(multi_horizon_results) * (1 + N_CLUSTERS)}개")
print(f"    - 호라이즌별 industry_features.csv: {len(multi_horizon_results)}개")
print(f"    - 호라이즌별 클러스터 CSV: {len(multi_horizon_results) * N_CLUSTERS}개")

print("\n" + "="*80)
print("완료")
print("="*80)


파이프라인 최종 요약
예측 기준일: 2026-01-16

멀티 호라이즌 예측:
  분석된 호라이즌: 7개
  호라이즌별 Top 섹터 수: 3개

호라이즌별 클러스터링 결과:

  [1d]:
    선정 섹터: Healthcare, Energy, Technology
    분석된 산업: 26개
    클러스터: 5개
    클러스터 분포: {'cluster_0': 2, 'cluster_1': 8, 'cluster_2': 2, 'cluster_3': 4, 'cluster_4': 10}

  [3d]:
    선정 섹터: Industrials, Consumer Cyclical, Energy
    분석된 산업: 34개
    클러스터: 5개
    클러스터 분포: {'cluster_1': 17, 'cluster_2': 6, 'cluster_3': 4, 'cluster_4': 7}

  [1w]:
    선정 섹터: Industrials, Consumer Cyclical, Communication Services
    분석된 산업: 33개
    클러스터: 5개
    클러스터 분포: {'cluster_1': 13, 'cluster_2': 3, 'cluster_3': 3, 'cluster_4': 14}

  [1m]:
    선정 섹터: Technology, Industrials, Communication Services
    분석된 산업: 30개
    클러스터: 5개
    클러스터 분포: {'cluster_0': 1, 'cluster_1': 5, 'cluster_2': 3, 'cluster_3': 3, 'cluster_4': 18}

  [1q]:
    선정 섹터: Communication Services, Financial Services, Industrials
    분석된 산업: 31개
    클러스터: 5개
    클러스터 분포: {'cluster_1': 11, 'cluster_2': 5, 'cluster_3': 4, 'cluster_4': 11}